![Logo](../logos/banner_colibri.png)

---
# 🏝️ Welcome to **colibri**'s playground!

Here, you will be able to test all the features of the package. Adapt parameters and run the cells corresponding to the function you want to test to see the result. Make sure you activate the conda environment `colibri` and the Jupyter kernel before starting. A quick description of the functionalities is provided. More technical information is available in docstrings of each function.

---
<br/>
<br/>

In [1]:
# Import colibri

import sys

sys.path.append("..")
import src


<br/>

**0. Set up your umbrella review** <br/>
Choose the scientific field you want to sythesise by setting up a search query. Select the platforms from which you want to get publications from.<br/>
*Nota bene:*
- *only scientific field studying Soil Organic Carbon is currently available*
- *only Web of Science platform is currently available*

In [2]:
search_query = "ts = (('meta*analysis' or 'systematic review') and ('soil organic carbon' or 'SOC' or 'soil organic matter' or 'SOM' or 'soil carbon'))"
platforms = ["WoS"]


<br/>

**1. Run the entire pipeline** <br/>
From scrapping publications to characterising their content. More details in README.md file.

In [4]:
src.wrapper.run_pipeline(search_query, platforms)

🐦 Welcome to the pipeline of colibri!

Phase I - Scrapping
You selected platform(s) Web of Science Core Collection to get the publications from.

Scrapping started at 2023-9-1_16-20-31 UTC.


Connected to Web of Science.


NoSuchElementException: Message: Unable to locate element: [id="onetrust-reject-all-handler"]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:188:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:506:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16


<br/>

**2. Scrapping** <br/>
Scrape publications on various platform with a specific search query. DOIs, titles, abstracts and keywords (when available) of each publication will be stored in Pickle files in folder `colibri/data/<yyyy>-<mm>-<dd>_<hh>-<mm>-<ss>` with the corresponding timestamp of your scrapping.

In [ ]:
src.scrapper.scrape(search_query, ["WoS"], src.glob_var.PLATFORM_MAP)